In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import glob
import random
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2023-01-07 19:41:37.626382: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2023-01-07 19:41:37.626564: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

print(tf.__version__)

Device: grpc://10.0.0.2:8470


2023-01-07 19:41:43.180194: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-07 19:41:43.182530: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2023-01-07 19:41:43.182561: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-07 19:41:43.182588: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (20b5f353cb42): /proc/driver/nvidia/version does not exist
2023-01-07 19:41:43.185588: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

Number of replicas: 8
2.4.1


In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_PATH = KaggleDatasets().get_gcs_path()
BATCH_SIZE = 64*strategy.num_replicas_in_sync
#BATCH_SIZE = 64 # for non-TPU use
IMAGE_SIZE = [180, 180]
EPOCHS = 50

In [4]:
filenames = tf.io.gfile.glob(str(GCS_PATH + '/chest_xray/train/*/*'))
filenames.extend(tf.io.gfile.glob(str(GCS_PATH + '/chest_xray/val/*/*')))
filenames.extend(tf.io.gfile.glob(str(GCS_PATH + '/chest_xray/test/*/*')))

print(f'The total number of files is {len(filenames)}')

2023-01-07 19:41:48.930051: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.
2023-01-07 19:41:49.050592: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.
2023-01-07 19:41:49.144660: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.
2023-01-07 19:41:49.233961: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.
2023-01-07 19:41:49.339280: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.


The total number of files is 5856


In [5]:
# split the data into 60:40 for train:test+val
train_files, val_test_files = train_test_split(filenames, test_size=0.4,random_state=18)

In [6]:
# split the val_test_files to 50:50 to have a final split of 60:20:20 of train:val:test
val_files, test_files = train_test_split(val_test_files, test_size=0.5, random_state=18)

In [7]:
pct_df = pd.DataFrame(columns = ['data_set', 'label', 'pct'], index = range(6))

totals = {'train':len(train_files), 'val':len(val_files), 'test':len(test_files)}

val_pneum_count = 0
val_norm_count = 0

re_pattern = '.*/(NORMAL|PNEUMONIA)/.*'

for file in val_files:
    re_match = re.match(re_pattern, file)
    if re_match[1] == 'NORMAL':
        val_norm_count += 1
    else:
        val_pneum_count += 1

pct_df.iloc[0, :] = ['val', 'NORMAL', round(100* val_norm_count / totals['val'], 1)]
pct_df.iloc[1, :] = ['val', 'PNEUMONIA', round(100* val_pneum_count / totals['val'], 1)]


test_pneum_count = 0
test_norm_count = 0
for file in test_files:
    re_match = re.match(re_pattern, file)
    if re_match[1] == 'NORMAL':
        test_norm_count += 1
    else:
        test_pneum_count += 1

pct_df.iloc[2, :] = ['test', 'NORMAL', round(100* test_norm_count / totals['test'], 1)]
pct_df.iloc[3, :] = ['test', 'PNEUMONIA', round(100* test_pneum_count / totals['test'], 1)]

train_pneum_count = 0
train_norm_count = 0
for file in train_files:
    re_match = re.match(re_pattern, file)
    if re_match[1] == 'NORMAL':
        train_norm_count += 1
    else:
        train_pneum_count += 1

pct_df.iloc[4, :] = ['train', 'NORMAL', round(100* train_norm_count / totals['train'], 1)]
pct_df.iloc[5, :] = ['train', 'PNEUMONIA', round(100* train_pneum_count / totals['train'], 1)]

In [8]:
PNEUM_COUNT = val_pneum_count + train_pneum_count + test_pneum_count
NORMAL_COUNT = val_norm_count + train_norm_count + test_norm_count
print(f'Normal: {NORMAL_COUNT}\nPneumonia: {PNEUM_COUNT}')

pct_df

Normal: 1583
Pneumonia: 4273


,data_set,label,pct
0,val,NORMAL,26.9
1,val,PNEUMONIA,73.1
2,test,NORMAL,27.6
3,test,PNEUMONIA,72.4
4,train,NORMAL,26.9
5,train,PNEUMONIA,73.1


In [9]:
train_list_ds = tf.data.Dataset.from_tensor_slices(train_files)
val_list_ds = tf.data.Dataset.from_tensor_slices(val_files)
test_list_ds = tf.data.Dataset.from_tensor_slices(test_files)

#view the file paths for training
print('----Training Files----')
for f in train_list_ds.take(5):
    print(f.numpy())



----Training Files----
b'gs://kds-c64d0f53ec9c4abe7fa90871ed8ddc3242cec4843db9c83a9f5d571d/chest_xray/train/PNEUMONIA/person509_virus_1025.jpeg'
b'gs://kds-c64d0f53ec9c4abe7fa90871ed8ddc3242cec4843db9c83a9f5d571d/chest_xray/train/PNEUMONIA/person671_bacteria_2564.jpeg'
b'gs://kds-c64d0f53ec9c4abe7fa90871ed8ddc3242cec4843db9c83a9f5d571d/chest_xray/train/PNEUMONIA/person957_virus_1629.jpeg'
b'gs://kds-c64d0f53ec9c4abe7fa90871ed8ddc3242cec4843db9c83a9f5d571d/chest_xray/test/PNEUMONIA/person81_bacteria_397.jpeg'
b'gs://kds-c64d0f53ec9c4abe7fa90871ed8ddc3242cec4843db9c83a9f5d571d/chest_xray/train/NORMAL/IM-0500-0001.jpeg'


2023-01-07 19:41:50.043702: W ./tensorflow/core/distributed_runtime/eager/destroy_tensor_handle_node.h:57] Ignoring an error encountered when deleting remote tensors handles: Invalid argument: Unable to find the relevant tensor remote_handle: Op ID: 28, Output num: 0
Additional GRPC error information from remote target /job:worker/replica:0/task:0:
:{"created":"@1673120510.040421621","description":"Error received from peer ipv4:10.0.0.2:8470","file":"external/com_github_grpc_grpc/src/core/lib/surface/call.cc","file_line":1056,"grpc_message":"Unable to find the relevant tensor remote_handle: Op ID: 28, Output num: 0","grpc_status":3}


In [10]:
TRAIN_IMG_COUNT = tf.data.experimental.cardinality(train_list_ds).numpy()
VAL_IMG_COUNT = tf.data.experimental.cardinality(val_list_ds).numpy()
TEST_IMG_COUNT = tf.data.experimental.cardinality(test_list_ds).numpy()

print(f'Train Image Count: {TRAIN_IMG_COUNT}\nVal Image Count: {VAL_IMG_COUNT}\nTest Image Count: {TEST_IMG_COUNT}')

Train Image Count: 3513
Val Image Count: 1171
Test Image Count: 1172


In [11]:
# will label as 1 if pneumonia otherwise 0
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2] == 'PNEUMONIA'

def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img, IMAGE_SIZE)

def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [12]:
train_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

val_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

test_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)



In [13]:
y_train = []
for image, label in train_ds.take(TRAIN_IMG_COUNT):
    y_train.append(label)
    
print('Training Counts')
print(f'Pneumonia count: {np.sum(y_train)}\nNormal count: {TRAIN_IMG_COUNT - np.sum(y_train)}')

2023-01-07 19:43:43.659332: W ./tensorflow/core/distributed_runtime/eager/destroy_tensor_handle_node.h:57] Ignoring an error encountered when deleting remote tensors handles: Invalid argument: Unable to find the relevant tensor remote_handle: Op ID: 3562, Output num: 1
Additional GRPC error information from remote target /job:worker/replica:0/task:0:
:{"created":"@1673120623.659233011","description":"Error received from peer ipv4:10.0.0.2:8470","file":"external/com_github_grpc_grpc/src/core/lib/surface/call.cc","file_line":1056,"grpc_message":"Unable to find the relevant tensor remote_handle: Op ID: 3562, Output num: 1","grpc_status":3}


Training Counts
Pneumonia count: 2568
Normal count: 945


In [14]:
y_val = []
for image, label in val_ds.take(VAL_IMG_COUNT):
    y_val.append(label)
    
print('Validation Counts')
print(f'Pneumonia count: {np.sum(y_val)}\nNormal count: {VAL_IMG_COUNT - np.sum(y_val)}')

Validation Counts
Pneumonia count: 856
Normal count: 315


In [15]:
y_test = []
for image, label in test_ds.take(TEST_IMG_COUNT):
    y_test.append(label)
    
print('Test Counts')
print(f'Pneumonia count: {np.sum(y_test)}\nNormal count: {TEST_IMG_COUNT - np.sum(y_test)}')



Test Counts


2023-01-07 19:45:21.905716: W ./tensorflow/core/distributed_runtime/eager/destroy_tensor_handle_node.h:57] Ignoring an error encountered when deleting remote tensors handles: Invalid argument: Unable to find the relevant tensor remote_handle: Op ID: 24663, Output num: 1
Additional GRPC error information from remote target /job:worker/replica:0/task:0:
:{"created":"@1673120721.905584672","description":"Error received from peer ipv4:10.0.0.2:8470","file":"external/com_github_grpc_grpc/src/core/lib/surface/call.cc","file_line":1056,"grpc_message":"Unable to find the relevant tensor remote_handle: Op ID: 24663, Output num: 1","grpc_status":3}


Pneumonia count: 849
Normal count: 323


In [16]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
            
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    
    ds = ds.repeat()
    
    ds = ds.batch(BATCH_SIZE)
    
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    
    return ds

In [17]:
train_ds = prepare_for_training(train_ds)
val_ds = prepare_for_training(val_ds)

In [18]:
weight_for_0 = (1 / NORMAL_COUNT)*(TRAIN_IMG_COUNT)/2.0
weight_for_1 = (1 / PNEUM_COUNT)*(TRAIN_IMG_COUNT)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}
print(f'Weight for class 0: {weight_for_0:.2f}')
print(f'Weight for class 1: {weight_for_1:.2f}')

Weight for class 0: 1.11
Weight for class 1: 0.41


In [19]:
# create some metrics for comparing on test results

def precision(y_test, y_pred):
    tp, fp = 0, 0
    
    for i in range(len(y_test)):
        if y_test[i] >= 0.5 and y_pred[i] >= 0.5:
            tp += 1
        elif y_test[i] < 0.5 and y_pred[i] >= 0.5:
            fp += 1
            
    return tp / (tp + fp)

def recall(y_test, y_pred):
    tp, fn = 0, 0
    
    for i in range(len(y_test)):
        if y_test[i] >= 0.5 and y_pred[i] >= 0.5:
            tp += 1
        elif y_test[i] >= 0.5 and y_pred[i] < 0.5:
            fn += 1
            
    return tp / (tp + fn)

def f1_score(y_test, y_pred):
    numerator = precision(y_test, y_pred) * recall(y_test, y_pred)
    denominator = precision(y_test, y_pred) + recall(y_test, y_pred)
    
    return 2 * (numerator / denominator)

## Fine Tune the Model

### Try different learning rate decays by adjusting the s value

In [20]:
# add exponential decay to try to improve fine tuning
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 ** (epoch / s)
    return exponential_decay_fn

In [21]:
# use learning rate that gave best results in the learning rate hyperparameter tuning
learning_rate = 0.06103
# use hidden_units that gave best results in the hidden units hyperparameter tuning
hidden_units = 4096
# use mini batch size that gave best results in the mini batch hyperparameter tuning
mini_batch = 32

# try 8 different decay rates
for _ in range(8):
    
    s = random.randint(10, 80)
    
    BATCH_SIZE = mini_batch*strategy.num_replicas_in_sync
    print(f'\n\nBatch Size: {BATCH_SIZE}')    
    print(f'Hidden Units: {hidden_units}')
    print(f'Learning Rate: {learning_rate}')
    print(f'1 Hidden Layer, {hidden_units} units')
    print(f'S value: {s}')
    
    with strategy.scope():
        
        base_model = tf.keras.applications.densenet.DenseNet201(
                        input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
                        include_top=False,
                        weights='imagenet'
                        )
        base_model.trainable = False
        
        
        
        x=base_model.output
        inputs = tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
        x = base_model(inputs, training=False)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dropout(0.3)(x)
        x = tf.keras.layers.Dense(hidden_units, activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.3)(x)
            
        outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
        model = tf.keras.Model(inputs, outputs)
        
        METRICS = [
                'accuracy',
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')
                ]
        
        model.compile(
                    optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=METRICS
                     )
    
       
        checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(f"densenet_{learning_rate}_{hidden_units}_{BATCH_SIZE}_1layer_{s}_xray_model.h5", save_best_only=True)
        early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)
        exponential_decay_fn = exponential_decay(learning_rate, s)
        lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)
    
        history = model.fit(
                            train_ds,
                            steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
                            epochs=EPOCHS,
                            validation_data=val_ds,
                            validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
                            class_weight=class_weight,
                            callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler]
                            )



Batch Size: 256
Hidden Units: 4096
Learning Rate: 0.06103
1 Hidden Layer, 4096 units
S value: 66
74842112/74836368 [==============================] - 0s 0us/step
Epoch 1/50
13/13 [==============================] - 139s 6s/step - loss: 5.6139 - accuracy: 0.7266 - precision: 0.8830 - recall: 0.7193 - val_loss: 125.1199 - val_accuracy: 0.6885 - val_precision: 0.9988 - val_recall: 0.5742
Epoch 2/50
13/13 [==============================] - 3s 222ms/step - loss: 3.0673 - accuracy: 0.9306 - precision: 0.9588 - recall: 0.9462 - val_loss: 3.3895 - val_accuracy: 0.9058 - val_precision: 0.9615 - val_recall: 0.9066
Epoch 3/50
13/13 [==============================] - 3s 225ms/step - loss: 0.7739 - accuracy: 0.9155 - precision: 0.9646 - recall: 0.9187 - val_loss: 4.8453 - val_accuracy: 0.7964 - val_precision: 1.0000 - val_recall: 0.7207
Epoch 4/50
13/13 [==============================] - 3s 224ms/step - loss: 0.4596 - accuracy: 0.9086 - precision: 0.9719 - recall: 0.9022 - val_loss: 0.8159 - val_a